# Voyages API Use Cases

## Setup
Install the Signal Ocean SDK:
```
pip install signal-ocean
```
And put your API key in a `SIGNAL_OCEAN_API_KEY` environment variable.

In [ ]:
from signal_ocean.voyages import VoyagesAPI
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme()

## Voyages API Use Cases


In [ ]:
api = VoyagesAPI()

### Get voyages

In [ ]:
vlcc_id = 84
date_from = datetime(2020, 1, 1)

In [ ]:
voyages = api.get_voyages(vessel_class_id=vlcc_id, date_from=date_from)

In [ ]:
voyages = pd.DataFrame(v.__dict__ for v in voyages)
events = pd.DataFrame(e.__dict__ for voyage_events in voyages['events'].dropna() for e in voyage_events)
historical_events = events[events['event_horizon']=='Historical']

In [ ]:
def get_voyage_load_area(voyage_events):
    return next((e.area_name_level0 for e in voyage_events or [] if e.purpose=='Load'), None)

voyages['load_area'] = voyages['events'].apply(get_voyage_load_area)

In [ ]:
def get_voyage_discharge_country(voyage_events):
    return next((e.country for e in reversed(voyage_events or []) if e.purpose=='Discharge'), None)

voyages['discharge_country'] = voyages['events'].apply(get_voyage_discharge_country)

In [ ]:
def get_voyage_load_country(voyage_events):
    return next((e.country for e in voyage_events or [] if e.purpose=='Load'), None)

voyages['load_country'] = voyages['events'].apply(get_voyage_load_country)

### Number of exporting voyages 

In [ ]:
voyages_exports_usg = voyages[(voyages['load_area']=='US Gulf')&(voyages['discharge_country']!='United States')]

In [ ]:
voyages_exports_usg.shape[0]

In [ ]:
voyages_exports_usg['discharge_country'].value_counts()

### Port Delays

In [ ]:
discharges_china = historical_events[(historical_events['country']=='China')&(historical_events['purpose']=='Discharge')].copy()
discharges_china['duration'] =  discharges_china['sailing_date'] - discharges_china['arrival_date']

In [ ]:
discharges_china['duration'].describe()

In [ ]:
discharges_china['duration_in_hours'] = discharges_china['duration'] / np.timedelta64(1, 'h')

In [ ]:
common_discharge_ports_china = discharges_china['port_name'].value_counts().head(8)
common_port_discharges_china = discharges_china[discharges_china['port_name'].isin(common_discharge_ports_china.index)]
sns.catplot(x="port_name", y="duration_in_hours", kind="box", data=common_port_discharges_china, aspect=2);

In [ ]:
discharges_china['arrival_month'] = discharges_china['arrival_date'].dt.tz_localize(None).dt.to_period('M').dt.to_timestamp()
sns.lineplot(data=discharges_china, x='arrival_month', y='duration_in_hours')
plt.xticks(rotation=90);

### Discharge destinations

In [ ]:
discharge_destinations_brazil = voyages[voyages['load_country']=='Brazil'].dropna(subset=['discharge_country'])

In [ ]:
discharge_destinations_brazil['discharge_country'].value_counts()

In [ ]:
sns.displot(discharge_destinations_brazil, x="start_date", hue="discharge_country", aspect=2);